### Init Environment

In [1]:
print('Installing torchprofile...')
!pip install torchprofile
print('All required packages have been successfully installed!')

Installing torchprofile...
All required packages have been successfully installed!


In [20]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os



### Data

In [59]:
annotation_train_path = './dataset/annotation_train.csv'
dataset_train_path = './dataset/data_train.npy'
annotation_test_path = './dataset/annotation_test.csv'
dataset_test_path = './dataset/data_test.npy'

class RunningDataset(Dataset):
    def __init__(self,annotation_path, dataset_path, transform=None):
        self.transform = transform
        self.labels = pd.read_csv(annotation_path, header=None)
        self.data = np.load(dataset_path)

        print (self.data.shape)
        print (self.labels.shape)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = torch.tensor(sample, dtype=torch.float32)
        label = self.labels.iloc[idx,0]
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            sample = self.transform(sample)
        
        return sample, label

train_dataset = RunningDataset(annotation_train_path, dataset_train_path)
test_dataset = RunningDataset(annotation_test_path, dataset_test_path)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=50,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=50,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
)

(56379, 90, 6)
(56379, 1)
(37530, 90, 6)
(37530, 1)


### Model

In [60]:
class MyModel(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)
    self.seq_len = 90

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    self.gru1 = nn.GRU(input_size=6, hidden_size=32, batch_first=True)
    self.relu1 = nn.ReLU()
    self.gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True)
    self.relu2 = nn.ReLU()
    add("conv", nn.Conv1d(32, 64, 2, stride=2))
    add("relu", nn.ReLU())
    add("pool", nn.MaxPool1d(4, padding=1))
    add("conv", nn.Conv1d(64, 128, 2, stride=1))
    add("relu", nn.ReLU())
    add("adaptivePoll",nn.AdaptiveAvgPool1d(1))
    add("bn", nn.BatchNorm1d(128, eps=1e-06))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(128, 4)
    # self.classifier = nn.Linear(128, 4)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x, _ = self.gru1(x)
    x = self.relu1(x)
    x, _ = self.gru2(x)
    x = self.relu2(x)
    x = x.reshape(-1, 32, self.seq_len)
    x = self.backbone(x)
    x = x.reshape(-1, 128)
    x = self.classifier(x)
    x = self.softmax(x)

    return x
  
model = MyModel().cuda()
## show model summary
print(model.eval())

MyModel(
  (gru1): GRU(6, 32, batch_first=True)
  (relu1): ReLU()
  (gru2): GRU(32, 32, batch_first=True)
  (relu2): ReLU()
  (backbone): Sequential(
    (conv0): Conv1d(32, 64, kernel_size=(2,), stride=(2,))
    (relu0): ReLU()
    (pool0): MaxPool1d(kernel_size=4, stride=4, padding=1, dilation=1, ceil_mode=False)
    (conv1): Conv1d(64, 128, kernel_size=(2,), stride=(1,))
    (relu1): ReLU()
    (adaptivePoll0): AdaptiveAvgPool1d(output_size=1)
    (bn0): BatchNorm1d(128, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classifier): Linear(in_features=128, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


### Training

In [55]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  callbacks = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Forward inference
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward propagation
    loss.backward()

    # Update optimizer and LR scheduler
    optimizer.step()
    scheduler.step()

    if callbacks is not None:
        for callback in callbacks:
            callback()

@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  verbose=True,
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False,
                              disable=not verbose):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Inference
    outputs = model(inputs)

    # Convert logits to class indices
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

# save checkpoint to local file
def save_checkpoint(
  model: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  epoch: int,
  accuracy: float,
  path: str,
) -> None:
  torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'scheduler': scheduler.state_dict(),
    'epoch': epoch,
    'accuracy': accuracy,
  }, path)


# load checkpoint from local file
def load_checkpoint(
  model: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  path: str,
) -> None:
  checkpoint = torch.load(path)

  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  scheduler.load_state_dict(checkpoint['scheduler'])

  return checkpoint['epoch'], checkpoint['accuracy']

# train model
def run(
  model: nn.Module,
  train_dataloader: DataLoader,
  valid_dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  num_epochs: int,
  checkpoint_path: str,
  verbose=True,
) -> None:
  best_accuracy = 0.0

  for epoch in tqdm(range(num_epochs)):
    # Train for one epoch
    train(model, train_dataloader, criterion, optimizer, scheduler)

    # Evaluate on validation set
    accuracy = evaluate(model, valid_dataloader, verbose)

    # Save checkpoint
    if accuracy > best_accuracy:
      best_accuracy = accuracy
      save_checkpoint(model, optimizer, scheduler, epoch, accuracy,
                      checkpoint_path)

    if verbose:
      print(f'Epoch: {epoch + 1:03d} | '
            f'accuracy = {accuracy:.2f} | '
            f'best_accuracy = {best_accuracy:.2f}')

In [63]:
torch.autograd.set_detect_anomaly(True)

num_epochs = 10
steps_per_epoch = len(train_dataloader)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=steps_per_epoch, gamma=0.1)
criterion = nn.CrossEntropyLoss()

checkpoint_path = "checkpoint.pth"
# if checkpoint exist then load checkpoint
if os.path.exists(checkpoint_path):
    epoch, accuracy = load_checkpoint(model, optimizer, scheduler, checkpoint_path)
    print(f"Load checkpoint from epoch {epoch} with accuracy {accuracy:.2f}")
    
run(model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, 10, checkpoint_path, verbose=True)

Load checkpoint from epoch 5 with accuracy 87.82


  0%|          | 0/10 [00:00<?, ?it/s]

train:   0%|          | 0/282 [00:00<?, ?it/s]

eval:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 001 | accuracy = 87.72 | best_accuracy = 87.72


...

eval:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 009 | accuracy = 87.76 | best_accuracy = 87.76


train:   0%|          | 0/282 [00:00<?, ?it/s]

eval:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 010 | accuracy = 87.71 | best_accuracy = 87.76
